## PEC-MDP Translator

The `PEC_Parser` is a Python-based module designed to convert Probabilistic Event Calculus (PEC) domains into Markov Decision Process (MDP) components. The parser utilizes natural language patterns to extract propositions and components from a given PEC domain string, enabling the translation process. This notebook demonstrates the functionality and flexibility of the PEC_Parser module. 
The demonstration is structured in three main parts:

* Introduction to the parser's functions, and how PEC propositions are translated into PEC-MDP components.

* Presentation of the pecmdp class, which encapsulates the translated PEC-MDP components in a full PEC-MDP for further manipulation.

* Practical application of the translator for probabilistic sampling, illustrating its utility in querying PEC-MDP models.

To highlight the versatility of the translator, the notebook includes several example domains with diverse properties. These examples serve to demonstrate how the PEC parser can handle a range of PEC domain specifications, from simple to complex. By the end of this demonstration, users will have a comprehensive understanding of how to leverage the PEC-MDP Translator to convert PEC domains into MDP components, facilitating advanced probabilistic reasoning and analysis in event-driven systems.

### Example PEC domains

* "PEC_example_1.txt" (Antibiotic): This domain models the interaction medicine with bacteria and rash. 

* "PEC_example_2.txt" (Tuberculosis): Another medical domain focusing on tuberculosis. It has a single fluent (tuberculosis) affected by two actions (exposure and reactivation).

* "PEC_example_3.txt" (Decay): This domain demonstrates a decay process where a single fluent starts as true and can be changed to false by an action at each time point. The probability of the fluent remaining true decays over time.

* "PEC_example_4.txt" (Tamagotchi): A more complex domain involving a virtual pet (Tamagotchi) with multiple fluents (hungry, thirsty, mood). This features a higher-dimensional domain with concurrent action-taking.

* "PEC_example_5.txt" (Coin and Dice): A simple, interpretable domain featuring a coin and a dice with conventional values. The domain includes actions for tossing the coin and rolling the die, which can change their respective values.

Domains 1-3 are sourced from the GitHub repository https://github.com/dasaro/pec-anglican and serve as standard examples of Probabilistic Event Calculus (PEC). Domains 4 and 5 are custom examples designed to showcase additional complexities and scenarios.

In [14]:
file_path = "PEC_example_5.txt"
file = open(file_path, "r")
domain_string = file.read()

### PEC Parser

The module `PEC_Parser` is comprised of two main classes `domain` and `PECMDP`. The first class `domain` contains various methods for encoding each PEC component into its PEC-MDP representation. The main methods which generate PEC-MDP components include:

* `initialise_all`

* `get_initial`

* `get_transition`

* `get_policy`


Firstly, the method `initialise_all` generates the fundamental constituents of a domain including its fluents, values, states, actions and time instants. The method must be called prior to initialising all other components of the PEC-MDP. It takes the domain string as input and extracts v-propositions to integer-encode fluents, values, and states. P-propositions are extracted to integer encode actions in the domain, including any combinations of actions that may occur simultaneously. `initialise_all` returns he dictionaries `fluent_dict`, `value_dict`, `state_dict`, and `action_dict`.

In [15]:
import PEC_Parser

# Instantiate a domain object
domain = PEC_Parser.domain()

# Compute fluents, values, states, and actions as dictionaries for domain object
domain.initialise_all(domain_string)

({'Coin': 0, 'Dice': 1},
 {'Heads': 0,
  'Tails': 1,
  'One': 2,
  'Two': 3,
  'Three': 4,
  'Four': 5,
  'Five': 6,
  'Six': 7},
 {(0, 2): 0,
  (0, 3): 1,
  (0, 4): 2,
  (0, 5): 3,
  (0, 6): 4,
  (0, 7): 5,
  (1, 2): 6,
  (1, 3): 7,
  (1, 4): 8,
  (1, 5): 9,
  (1, 6): 10,
  (1, 7): 11},
 {'TossCoin': 0, 'TossDice': 1, 'TossCoin and TossDice': 2, 'null': 3})

Dictionaries of PEC components may also be accessed through class variables of the domain object. Dictionaries for fluents, values, and actions display a mapping between their PEC representation and integer encoding. The state dictionary, on the other hand, displays a mapping between vector state representations and integer state representations. Specific fluent state values may be retrieved by referring to fluent and value dictionaries.

In [16]:
fluent_dict = domain.fluent_dict
value_dict = domain.value_dict
state_dict = domain.state_dict
action_dict = domain.action_dict

print(f"Fluents: {fluent_dict}")
print(f"Values: {value_dict}")
print(f"States: {state_dict}")
print(f"Actions: {action_dict}")

Fluents: {'Coin': 0, 'Dice': 1}
Values: {'Heads': 0, 'Tails': 1, 'One': 2, 'Two': 3, 'Three': 4, 'Four': 5, 'Five': 6, 'Six': 7}
States: {(0, 2): 0, (0, 3): 1, (0, 4): 2, (0, 5): 3, (0, 6): 4, (0, 7): 5, (1, 2): 6, (1, 3): 7, (1, 4): 8, (1, 5): 9, (1, 6): 10, (1, 7): 11}
Actions: {'TossCoin': 0, 'TossDice': 1, 'TossCoin and TossDice': 2, 'null': 3}


#### Initial distribution of states

The method `get_initial` returns a tuple containing a list of initial states and a list of their corresponding probabilities. The state with index $i$ in the first list occurs with the probability given by the $i^{th}$ element of the second list.

In [17]:
initial_distribution = domain.get_initial(domain_string)
print(initial_distribution)

([3, 9], [0.8, 0.2])


#### Transition function

`get_transition` returns a matrix for each action of shape $s_n \times s_n$, where $s_n$ is the number of states. The element $p_{i,j}$ refers to the probability of transitioning from the $i^{th}$ state to the $j^{th}$ state. For the "null" action, the identify matrix gives the state-to-state transitions.

In [18]:
transition_matrix = domain.get_transition(domain_string)
for action, action_int in action_dict.items():
    print(f"{action}:")
    for row in transition_matrix[action_int]:
        print(row)

TossCoin:
[0.51, 0, 0, 0, 0, 0, 0.49, 0, 0, 0, 0, 0]
[0, 0.51, 0, 0, 0, 0, 0, 0.49, 0, 0, 0, 0]
[0, 0, 0.51, 0, 0, 0, 0, 0, 0.49, 0, 0, 0]
[0, 0, 0, 0.51, 0, 0, 0, 0, 0, 0.49, 0, 0]
[0, 0, 0, 0, 0.51, 0, 0, 0, 0, 0, 0.49, 0]
[0, 0, 0, 0, 0, 0.51, 0, 0, 0, 0, 0, 0.49]
[0.49, 0, 0, 0, 0, 0, 0.51, 0, 0, 0, 0, 0]
[0, 0.49, 0, 0, 0, 0, 0, 0.51, 0, 0, 0, 0]
[0, 0, 0.49, 0, 0, 0, 0, 0, 0.51, 0, 0, 0]
[0, 0, 0, 0.49, 0, 0, 0, 0, 0, 0.51, 0, 0]
[0, 0, 0, 0, 0.49, 0, 0, 0, 0, 0, 0.51, 0]
[0, 0, 0, 0, 0, 0.49, 0, 0, 0, 0, 0, 0.51]
TossDice:
[0.2, 0.16, 0.16, 0.16, 0.16, 0.16, 0, 0, 0, 0, 0, 0]
[0.16, 0.2, 0.16, 0.16, 0.16, 0.16, 0, 0, 0, 0, 0, 0]
[0.16, 0.16, 0.2, 0.16, 0.16, 0.16, 0, 0, 0, 0, 0, 0]
[0.16, 0.16, 0.16, 0.2, 0.16, 0.16, 0, 0, 0, 0, 0, 0]
[0.16, 0.16, 0.16, 0.16, 0.2, 0.16, 0, 0, 0, 0, 0, 0]
[0.16, 0.16, 0.16, 0.16, 0.16, 0.2, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0.2, 0.16, 0.16, 0.16, 0.16, 0.16]
[0, 0, 0, 0, 0, 0, 0.16, 0.2, 0.16, 0.16, 0.16, 0.16]
[0, 0, 0, 0, 0, 0, 0.16, 0.16, 0

#### Policy

`get_policy` returns a probability matrix for each timestep. Each matrix is of size $(a_n+1) \times s_n$ where $a_n + 1 $ is the number of actions including the additional null-action $A_\emptyset$ and $s_n$ is the number of states. Thus, element $p_{i,j}$ from the $n^{th}$ matrix refers to probability that action $j$ will be taken in state $i$ at time instant $n$. The class variable `max_instant` denotes the maximum instant in the domain, or the time instant after the last action is performed.

In [19]:
policy_matrix = domain.get_policy(domain_string)
for row in range(len(policy_matrix)):
    print(f"{row}:", policy_matrix[row])

steps_n = domain.max_instant
print(f"Max steps: {steps_n}")

0: [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
1: [[0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875], [0.5625, 0.0625, 0.1875, 0.1875]]
2: [[0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875], [0.0625, 0.5625, 0.1875, 0.1875]]
3: [

### PEC-MDP

The `pecmdp` class of the module takes the parsed components of a domain and packages them into a full PECMDP with the following methods:

* `step`: Performs a step of agent-environment interaction based on a given action, updating the current state through transition probabilities.

* `reset`: Resets the PECMDP by setting the current state to one of those detailed by the initial distribution of states.

* `sample_trajectory`: samples a trace of the PECMDP by taking steps in the environment according to the previously defined policy and storing a trajectory as a sequence of states.

The MDP object is first initialised with a domain's state and action encodings, the transition matrix, and the initial distribution.

In [20]:
states = list(state_dict.values())
actions = list(action_dict.values())

mdp = PEC_Parser.pecmdp(states, actions, transition_matrix, initial_distribution)

A sampled trajectory returns states at each time instant. It takes the maximum time instant so that it resets after each sequence reaches this number of steps. Note that the `translate_int_to_lit` function of the module may be used to map encoded states back into a PEC natural language representation.

In [21]:
# Sample a trajectory
n_steps = steps_n # Steps retrieved from policy previously (domain.max_instant)
trajectory = mdp.sample_trajectory(n_steps, policy_matrix)

print(f"Sampled trajectory: {trajectory}")
print(f"Sampled trajectory: {[domain.translate_int_to_lit(s) for s in trajectory]}")

Sampled trajectory: [9, 9, 9, 11]
Sampled trajectory: ['Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Six']


#### Sampling and PEC Querying

A simple application of this module is for probabilistic sampling. By sampling a high number of trajectories, the probability of fluent states and values at specific instants may be estimated.

In [22]:
sample_n = 10000
sampled_trajectories = []

for sample_ in range(sample_n):
    trajectory = mdp.sample_trajectory(n_steps, policy_matrix)
    sampled_trajectories.append(trajectory)

Trajectories may be translated back into natural language for interpretability through the `translate_int_to_lit` function of the `domain` class.

In [23]:
for i in sampled_trajectories[:10]:
    print([domain.translate_int_to_lit(s) for s in i])

['Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Six']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Three']
['Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Four']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Tails,Dice=Five', 'Coin=Tails,Dice=Six']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=One']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Tails,Dice=Four', 'Coin=Tails,Dice=Three']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=One']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=One', 'Coin=Heads,Dice=Five']
['Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four', 'Coin=Heads,Dice=Four']


The sampled trajectories can be queried to determine the probability of specific fluent values holding at a given time instant. Queries consist of two parts:

* Time instant (_type_:  integer)
* Partial or complete fluent state (_type_: string)

In [24]:
query_time = 2 # From 0 to max time instant
query_literal = "Coin=Heads, Dice=Three"

states_associated = set(domain.partial_to_states(query_literal))
sampled_states = [trajectory_[query_time] for trajectory_ in sampled_trajectories if trajectory_[query_time] in states_associated]
    
answer = len(sampled_states) / sample_n
print(answer)

0.0215


#### Additional Informtation on Available Functions

In [25]:
help(domain.partial_to_states)

Help on method partial_to_states in module PEC_Parser:

partial_to_states(partial_fluent) method of PEC_Parser.domain instance
    _summary_
    
    Args:
        partial_fluent (_type_): string containing partial fluent state



In [26]:
help(domain.update_fluent_state)

Help on method update_fluent_state in module PEC_Parser:

update_fluent_state(current_state, partial_fluent) method of PEC_Parser.domain instance
    _summary_
    
    Args:
        current_state (_type_): integer encoded state (type integer and not tuple)
        partial_fluent (_type_): string containing partial fluent state

